# Installation

## Env setup

In [1]:
!conda env create -f environment_dev.yml
!conda activate AutoPrompt

WARNING conda.models.version:get_matcher(556): Using .* with relational operator is superfluous and deprecated and will be removed in a future version of conda. Your spec was 1.8.0.*, but conda is ignoring the .* and treating it as 1.8.0
done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 24.11.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.11.0



datasets-3.1.0       | 325 KB    |                                       |   0% 
pyarrow-core-18.0.0  | 3.7 MB    |                                       |   0% 

snappy-1.2.1         | 35 KB     |                                       |   0% 


aws-c-sdkutils-0.2.1 | 49 KB     |                                       |   0% 



orc-2.0.3            | 428 KB    |                                       |   0% 




libedit-3.1.20191231 | 94 

In [2]:
!pip install -r requirements.txt

  Obtaining dependency information for langchain==0.1.9 from https://files.pythonhosted.org/packages/45/05/2d5c38c7fa1a091a61e018070b6878410d256be577e5154843d339e59b2c/langchain-0.1.9-py3-none-any.whl.metadata
  Using cached langchain-0.1.9-py3-none-any.whl.metadata (13 kB)
  Obtaining dependency information for openai==1.1.0 from https://files.pythonhosted.org/packages/77/b4/8de305dd47c13e3ac2e8eef6dc1ebc6f0884d4ac67af9f86d612c949ab63/openai-1.1.0-py3-none-any.whl.metadata
  Using cached openai-1.1.0-py3-none-any.whl.metadata (16 kB)
  Obtaining dependency information for tiktoken==0.5.1 from https://files.pythonhosted.org/packages/fb/2a/3d02ef030f387c373acbeca6d5a2307405a1da735285ec12a9ed0b6302ea/tiktoken-0.5.1-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Using cached tiktoken-0.5.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.6 kB)
  Obtaining dependency information for transformers==4.35.2 from https://files.pythonhosted.org/packages/12/dd/f17b11a93a9ca27728e12512d167eb1281c151c4c

## Human Anotator setup

In [3]:
!pip install argilla

In [5]:
import argilla as rg

## Weights & Bias setup

In [ ]:
## copy this into the terminal
# !pip install wandb
# !wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
Aborted!


In [3]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset

    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dansashadan. Use `wandb login --relogin` to force relogin


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


acc,▁▅▇▇▇███
loss,█▅▅▄▁▂▁▂
acc,0.74977
loss,0.24823


# Classification run

In [5]:
from optimization_pipeline import OptimizationPipeline
from utils.config import load_yaml, override_config
import argparse


In [14]:

# General Training Parameters
parser = argparse.ArgumentParser()

parser.add_argument('--basic_config_path', default='config/config_default.yml', type=str, help='Configuration file path')
parser.add_argument('--batch_config_path', default='',
                    type=str, help='Batch classification configuration file path')
parser.add_argument('--prompt',
                    default='',
                    required=False, type=str, help='Prompt to use as initial.')
parser.add_argument('--task_description',
                    default='',
                    required=False, type=str, help='Describing the task')
parser.add_argument('--load_path', default='', required=False, type=str, help='In case of loading from checkpoint')
parser.add_argument('--output_dump', default='dump', required=False, type=str, help='Output to save checkpoints')
parser.add_argument('--num_steps', default=10, type=int, help='Number of iterations')

opt = parser.parse_args([])

if opt.batch_config_path == '':
    # load the basic configuration using load_yaml
    config_params = load_yaml(opt.basic_config_path)
else:
    # override the basic configuration with the batch configuration
    config_params = override_config(opt.batch_config_path, config_file=opt.basic_config_path)

if opt.task_description == '':
    task_description = input("Describe the task: ")
else:
    task_description = opt.task_description

if opt.prompt == '':
    initial_prompt = input("Initial prompt: ")
else:
    initial_prompt = opt.prompt

# Initializing the pipeline
pipeline = OptimizationPipeline(config_params, task_description, initial_prompt, output_path=opt.output_dump)
if (opt.load_path != ''):
    pipeline.load_state(opt.load_path)
best_prompt = pipeline.run_pipeline(opt.num_steps)
print('\033[92m' + 'Calibrated prompt score:', str(best_prompt['score']) + '\033[0m')
print('\033[92m' + 'Calibrated prompt:', best_prompt['prompt'] + '\033[0m')



wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Total usage,▁▂▂▃▄▄▅▅▅▅▆▆▇▇▇██
score,▁███▂█▄█▃▇█▅▄▆█▅▆
Total usage,0.32537
score,0.76


Starting step 0
Dataset is empty generating initial samples


Processing samples: 100%|██████████| 10/10 [00:19<00:00,  1.99s/it]


...

Previous prompt score:
0.72
#########

Get new prompt:
# Balanced Hiring Plan Classification Task

Classify the hiring plan by assessing whether it meets the sector-specific needs for top management roles and fits within the financial framework of the company. Take into account both the adherence to the budget and the correct prioritization of sector-specific roles, ensuring that neither is neglected.

**Evaluation Criteria:**
- Verify that the hiring plan provides competitive salaries for sector-specific top management roles without exceeding the company's financial boundaries.
- Confirm that sector-specific management roles are not only included but are also given the right level of priority within the hiring plan.
- Evaluate the total cost of the hiring plan, ensuring it does not surpass the financial limit set by the company.

**Classification Labels:**
- Yes: The hiring plan aligns with the financial and sector-specific requirements.
- No: The hiring plan fails to align with eithe

Processing samples: 0it [00:00, ?it/s]
Processing samples: 100%|██████████| 50/50 [00:05<00:00,  9.47it/s]


Previous prompt score:
0.44
#########

Get new prompt:
# Balanced Hiring Plan Analysis Task

Determine whether the hiring plan presents a well-balanced approach to hiring sector-specific top management roles without exceeding the company's financial limits. Assess whether the plan takes into account realistic sector-specific salary ranges, prioritizes the hiring order of top management roles according to the sector's needs, and ensures the total cost is within the financial budget. Correct any model biases towards negative classification, and accurately identify plans that effectively balance top talent acquisition and budgetary constraints.

**Evaluation Criteria:**
- The hiring plan's adherence to the company's financial budget, considering the sector-specific salary scales for top management positions.
- Effective prioritization of sector-specific top management roles in the hiring plan.
- A clear balance between the hiring of indispensable management talent and the overall financia

Processing samples: 0it [00:00, ?it/s]
Processing samples: 100%|██████████| 50/50 [00:05<00:00,  8.79it/s]


Calibrated prompt score: 0.95
Calibrated prompt: # Hiring Plan Financial and Management Evaluation Task

Evaluate if the hiring plan adheres to the company's financial constraints and properly prioritizes top management roles within the context of the company's sector. Incorporate specific salary information to ensure the total yearly cost does not exceed the company's stated yearly balance. Confirm that top management roles are not only included but are prioritized appropriately in the hiring sequence.

**Evaluation Criteria:**
- The hiring plan's total yearly cost, including specific salaries, remains within the company's declared yearly balance.
- Top management roles are clearly prioritized in the hiring order and adequately represented, according to the sector's needs.

**Classification Labels:**
- Yes: The hiring plan fulfills both the financial constraint adherence and the prioritization of top management roles.
- No: The hiring plan fails to meet the financial constraints or do

# Generation run

In [1]:
## test code
from optimization_pipeline import OptimizationPipeline
from utils.config import load_yaml, modify_input_for_ranker, validate_generation_config, override_config
import argparse
import os
from estimator.estimator_llm import LLMEstimator

# General Training Parameters
parser = argparse.ArgumentParser()

parser.add_argument('--generation_config_path', default='config/config_diff/config_generation.yml', type=str, help='Configuration file path')
parser.add_argument('--ranker_config_path', default='config/config_diff/config_ranking.yml', type=str, help='Configuration file path')

parser.add_argument('--task_description',
                    default='',
                    required=False, type=str, help='Describing the task')
parser.add_argument('--prompt',
                    default='',
                    required=False, type=str, help='Prompt to use as initial.')
parser.add_argument('--load_dump', default='', required=False, type=str, help='In case of loading from checkpoint')
parser.add_argument('--output_dump', default='dump', required=False, type=str, help='Output to save checkpoints')
parser.add_argument('--num_ranker_steps', default=10, type=int, help='Number of iterations')
parser.add_argument('--num_generation_steps', default=20, type=int, help='Number of iterations')

opt = parser.parse_args([])

ranker_config_params = override_config(opt.ranker_config_path)
generation_config_params = override_config(opt.generation_config_path)
validate_generation_config(ranker_config_params, generation_config_params)

if opt.task_description == '':
    task_description = input("Describe the task: ")
else:
    task_description = opt.task_description

if opt.prompt == '':
    initial_prompt = input("Initial prompt: ")
else:
    initial_prompt = opt.prompt

ranker_pipeline = OptimizationPipeline(ranker_config_params, output_path=os.path.join(opt.output_dump, 'ranker'))
if opt.load_dump != '':
    ranker_pipeline.load_state(os.path.join(opt.load_dump, 'ranker'))
    ranker_pipeline.predictor.init_chain(ranker_config_params.dataset.label_schema)

if (ranker_pipeline.cur_prompt is None) or (ranker_pipeline.task_description is None):
    ranker_mod_prompt, ranker_mod_task_desc = modify_input_for_ranker(
        ranker_config_params, task_description, initial_prompt)
    ranker_pipeline.cur_prompt = ranker_mod_prompt
    ranker_pipeline.task_description = ranker_mod_task_desc

# Adding debug statements before running the pipeline
print("\n=== Running Ranker Pipeline ===")
print("Current Prompt:", ranker_pipeline.cur_prompt)
print("Task Description:", ranker_pipeline.task_description)

best_prompt = ranker_pipeline.run_pipeline(opt.num_ranker_steps)

# Adding debug statements after running the pipeline
print("\n=== Ranker Pipeline Completed ===")
print("Best Prompt from Ranker Pipeline:", best_prompt)

generation_config_params.eval.function_params = ranker_config_params.predictor.config
generation_config_params.eval.function_params.instruction = best_prompt['prompt']
generation_config_params.eval.function_params.label_schema = ranker_config_params.dataset.label_schema

generation_pipeline = OptimizationPipeline(
    generation_config_params, task_description, initial_prompt,
    output_path=os.path.join(opt.output_dump, 'generator'))
if opt.load_dump != '':
    generation_pipeline.load_state(os.path.join(opt.load_dump, 'generator'))

# Adding debug statements before running the generation pipeline
print("\n=== Running Generation Pipeline ===")
print("Instruction:", generation_config_params.eval.function_params.instruction)
print("Label Schema:", generation_config_params.eval.function_params.label_schema)

best_generation_prompt = generation_pipeline.run_pipeline(opt.num_generation_steps)

print('\033[92m' + 'Calibrated prompt score:', str(best_generation_prompt['score']) + '\033[0m')
print('\033[92m' + 'Calibrated prompt:', best_generation_prompt['prompt'] + '\033[0m')

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: dansashadan. Use `wandb login --relogin` to force relogin


/Users/ogc2020/Documents/GitHub/AutoPrompt/utils/llm_chain.py:162: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  self.chain = LLMChain(llm=self.llm, prompt=self.prompt)
/Users/ogc2020/Documents/GitHub/AutoPrompt/utils/config.py:125: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  task_result = task_llm_chain(


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:47                                                                                   │
│                                                                                                  │
│   44 │   ranker_pipeline.predictor.init_chain(ranker_config_params.dataset.label_schema)         │
│   45                                                                                             │
│   46 if (ranker_pipeline.cur_prompt is None) or (ranker_pipeline.task_description is None):      │
│ ❱ 47 │   ranker_mod_prompt, ranker_mod_task_desc = modify_input_for_ranker(                      │
│   48 │   │   ranker_config_params, task_description, initial_prompt)                             │
│   49 │   ranker_pipeline.cur_prompt = ranker_mod_prompt                                          │
│   50 │   ranker_pipeline.task_description = ranker_mod_task_desc                                 │
│                                                                                                  │
│ /Users/ogc2020/Documents/GitHub/AutoPrompt/utils/config.py:125 in modify_input_for_ranker        │
│                                                                                                  │
│   122 │                                                                                          │
│   123 │   llm = get_llm(config.llm)                                                              │
│   124 │   task_llm_chain = LLMChain(llm=llm, prompt=task_desc_setup)                             │
│ ❱ 125 │   task_result = task_llm_chain(                                                          │
│   126 │   │   {"task_description": task_description})                                            │
│   127 │   mod_task_desc = task_result['text']                                                    │
│   128 │   logging.info(f"Task description modified for ranking to: \n{mod_task_desc}")           │
│                                                                                                  │
│ /Users/ogc2020/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:182 in  │
│ warning_emitting_wrapper                                                                         │
│                                                                                                  │
│   179 │   │   │   if not warned and not is_caller_internal():                                    │
│   180 │   │   │   │   warned = True                                                              │
│   181 │   │   │   │   emit_warning()                                                             │
│ ❱ 182 │   │   │   return wrapped(*args, **kwargs)                                                │
│   183 │   │                                                                                      │
│   184 │   │   async def awarning_emitting_wrapper(*args: Any, **kwargs: Any) -> Any:             │
│   185 │   │   │   """Same as warning_emitting_wrapper, but for async functions."""               │
│                                                                                                  │
│ /Users/ogc2020/anaconda3/lib/python3.11/site-packages/langchain/chains/base.py:389 in __call__   │
│                                                                                                  │
│   386 │   │   │   "run_name": run_name,                                                          │
│   387 │   │   }                                                                                  │
│   388 │   │                                                                                      │
│ ❱ 389 │   │   return self.invoke(                                                                │
│   390 │   │   │   inputs,                                                                        │
│   391 │   │   │   cast(RunnableConfig, {k: v for k, v in co

In [ ]:
## original code
from optimization_pipeline import OptimizationPipeline
from utils.config import load_yaml, modify_input_for_ranker, validate_generation_config, override_config
import argparse
import os
from estimator.estimator_llm import LLMEstimator
# General Training Parameters
parser = argparse.ArgumentParser()

parser.add_argument('--generation_config_path', default='config/config_diff/config_generation.yml', type=str, help='Configuration file path')
parser.add_argument('--ranker_config_path', default='config/config_diff/config_ranking.yml', type=str, help='Configuration file path')

parser.add_argument('--task_description',
                    default='',
                    required=False, type=str, help='Describing the task')
parser.add_argument('--prompt',
                    default='',
                    required=False, type=str, help='Prompt to use as initial.')
parser.add_argument('--load_dump', default='', required=False, type=str, help='In case of loading from checkpoint')
parser.add_argument('--output_dump', default='dump', required=False, type=str, help='Output to save checkpoints')
parser.add_argument('--num_ranker_steps', default=5, type=int, help='Number of iterations')
parser.add_argument('--num_generation_steps', default=20, type=int, help='Number of iterations')

opt = parser.parse_args([])


ranker_config_params = override_config(opt.ranker_config_path)
generation_config_params = override_config(opt.generation_config_path)
validate_generation_config(ranker_config_params, generation_config_params)

if opt.task_description == '':
    task_description = input("Describe the task: ")
else:
    task_description = opt.task_description

if opt.prompt == '':
    initial_prompt = input("Initial prompt: ")
else:
    initial_prompt = opt.prompt

ranker_pipeline = OptimizationPipeline(ranker_config_params, output_path=os.path.join(opt.output_dump, 'ranker'))
if opt.load_dump != '':
    ranker_pipeline.load_state(os.path.join(opt.load_dump, 'ranker'))
    ranker_pipeline.predictor.init_chain(ranker_config_params.dataset.label_schema)

if (ranker_pipeline.cur_prompt is None) or (ranker_pipeline.task_description is None):
    ranker_mod_prompt, ranker_mod_task_desc = modify_input_for_ranker(ranker_config_params, task_description,
                                                                      initial_prompt)
    ranker_pipeline.cur_prompt = ranker_mod_prompt
    ranker_pipeline.task_description = ranker_mod_task_desc

best_prompt = ranker_pipeline.run_pipeline(opt.num_ranker_steps)
generation_config_params.eval.function_params = ranker_config_params.predictor.config
generation_config_params.eval.function_params.instruction = best_prompt['prompt']
generation_config_params.eval.function_params.label_schema = ranker_config_params.dataset.label_schema


generation_pipeline = OptimizationPipeline(generation_config_params, task_description, initial_prompt,
                                           output_path=os.path.join(opt.output_dump, 'generator'))
if opt.load_dump != '':
    generation_pipeline.load_state(os.path.join(opt.load_dump, 'generator'))
best_generation_prompt = generation_pipeline.run_pipeline(opt.num_generation_steps)
print('\033[92m' + 'Calibrated prompt score:', str(best_generation_prompt['score']) + '\033[0m')
print('\033[92m' + 'Calibrated prompt:', best_generation_prompt['prompt'] + '\033[0m')


In [17]:
print("Annotations in dataset:", self.dataset['annotation'].unique())
print("Label schema:", self.label_schema)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 print("Annotations in dataset:", self.dataset['annotation'].unique())                        │
│   2 print("Label schema:", self.label_schema)                                                    │
│   3                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'self' is not defined